# Importing libraries

In [1]:
import warnings
warnings.simplefilter(action='ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import sys
sys.path.append('/home/chayan/UMINTFS/')

import muon as mu
import numpy as np
import pandas as pd
import scanpy as sc
import pickle as pkl
import seaborn as sns
from anndata import AnnData
from matplotlib import pyplot as plt
from utills import datasets as ds

# Loading Data

In [2]:
dataname = 'pbmc10k_atac'
x1, x2, y = ds.LoadData(dataname)

In [3]:
adata_rna = AnnData(X=x1, obs=y)
adata_atac = AnnData(X=x2, obs=y)

In [4]:
mdata = mu.MuData({'rna': adata_rna, 'atac': adata_atac})
mdata

MuData object with n_obs × n_vars = 7563 × 22456
  2 modalities
    rna:	7563 x 3026
      obs:	'celltype'
    atac:	7563 x 19430
      obs:	'celltype'

In [5]:
mu.pp.intersect_obs(mdata)
mdata

MuData object with n_obs × n_vars = 7563 × 22456
  2 modalities
    rna:	7563 x 3026
      obs:	'celltype'
    atac:	7563 x 19430
      obs:	'celltype'

# Run MOFA+

In [6]:
mu.tl.mofa(mdata, outfile=dataname+"_MUON.hdf5")


        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
Loaded view='rna' group='group1' with N=7563 samples and D=3026 features...
Loaded view='atac' group='group1' with N=7563 samples and D=19430 features...


Model options:
- Automatic Relevance Determination prior on the factors: True
- Automatic Relevance Determination prior on the weights: True
- Spike-and-slab prior on the factors: False
- Spike-and-slab prior on the weights: 

In [7]:
mdata.obsm['X_mofa'].shape

(7563, 10)

# Find neighbours using the integrated assay

In [8]:
sc.pp.neighbors(mdata, use_rep="X_mofa")
mdata

MuData object with n_obs × n_vars = 7563 × 22456
  obsm:	'X_mofa'
  varm:	'LFs'
  obsp:	'distances', 'connectivities'
  2 modalities
    rna:	7563 x 3026
      obs:	'celltype'
    atac:	7563 x 19430
      obs:	'celltype'

# Find clusters using Leiden

In [9]:
sc.tl.leiden(mdata, key_added='leiden_joint')
mdata

MuData object with n_obs × n_vars = 7563 × 22456
  obs:	'leiden_joint'
  obsm:	'X_mofa'
  varm:	'LFs'
  obsp:	'distances', 'connectivities'
  2 modalities
    rna:	7563 x 3026
      obs:	'celltype'
    atac:	7563 x 19430
      obs:	'celltype'

In [10]:
mdata['rna'].obs['leiden_joint'] = mdata.obs.leiden_joint
mdata['atac'].obs['leiden_joint'] = mdata.obs.leiden_joint

# Find top features (Leiden cell-type specific)

In [11]:
sc.tl.rank_genes_groups(mdata['rna'], 'leiden_joint', method='t-test_overestim_var')

# Saving the results

In [12]:
result = {}
result['rna'] = mdata['rna'].uns['rank_genes_groups']
result['rna']['genes'] = result['rna']['names']
groups = result['rna']['names'].dtype.names

top_Genes = [set(result['rna']['genes'][j][:10]) for j in groups]
top_Genes_dict = dict(enumerate(top_Genes))
top_Genes = set().union(*top_Genes)

In [13]:
x1[top_Genes].to_csv('MUON_Supervised_'+dataname+'_rna.csv')
mdata['rna'].obs['leiden_joint'].to_csv('MUON_Leiden_lables_'+dataname+'.csv')

In [14]:
with open('MUON_Supervised_'+dataname+'.pkl', 'wb') as f:
        pkl.dump(top_Genes_dict, f)

In [15]:
mdata.write(dataname+"_mudata_muon.h5mu")